In [1]:
import sys
from memory_profiler import profile
from collections import defaultdict
import collections
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from collections import OrderedDict
import os
import torch
import pandas as pd
import time
import pickle
import new_alternate_single_svm as nalsvm
from mkl_data_processing import storage_location, cross_validation_results_location, evaluate_predictions, oos_results_location, return_cross_val_symbol_path, model_dates_list
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier
from mkl_model_cross_validation import load_pickled_in_filename
# evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
# MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, \
    KOMD  # KOMD is not a MKL algorithm but a simple kernel machine like the SVM

In [2]:


# pick the symbols you want to try out of sample
# pick the model dates- this is important as you need to calculate the out of sample "forward" dates
# pick label
alternate_label = 'LabelsAlternateFour'
# index of label - this can be redundant
alternate_label_idx = list(nalsvm.labels_pickle_files).index(alternate_label)
print(alternate_label)
# clean data location
# oos_results_dict = defaultdict(dict)

LabelsAlternateFour


In [27]:
os.listdir('/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean')

['AAL.L',
 'BATS.L',
 'CCL.L',
 'CPG.L',
 'CV_Results',
 'DMGOa.L',
 'ECM.L',
 'GKN.L',
 'OOS_Results',
 'PRU.L',
 'PSON.L',
 'RB.L',
 'RDSa.L',
 'RDSb.L',
 'RSA.L',
 'RTO.L',
 'SDR.L',
 'SHP.L',
 'SMIN.L',
 'STAN.L']

In [28]:
os.listdir('/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results')

['AAL.L',
 'BATS.L',
 'ECM.L',
 'GKN.L',
 'PSON.L',
 'RDSa.L',
 'RDSb.L',
 'RSA.L',
 'RTO.L',
 'SHP.L',
 'STAN.L']

In [10]:

for symbol in ['RSA.L']:
    clean_data_location = storage_location(symbol)
    print(symbol)
    # model dates list
    model_dates = model_dates_list(return_cross_val_symbol_path(symbol))
    # location of data -->dataDrive, Clean Data Storage, and label.
    pkl_file = load_pickled_in_filename(
        os.path.join(clean_data_location, os.listdir(clean_data_location)[alternate_label_idx]))
    date_keys = list(pkl_file.keys())
    # list of out of sample dates
    results_dict = defaultdict(dict)
    for model_idx, model_date in enumerate(model_dates):
        
        #dictionaries to store everything
        

        
        print(model_idx)
        
        forward_dates = nalsvm.forwardDates(date_keys, model_date)

        print('---------------> Doing Model Date:', model_date)
        try:
            mkl_dict = collections.defaultdict(dict)
            average_dict = collections.defaultdict(dict)
            # put the features in a tensor format
            Xtr = normalization(rescale_01(torch.Tensor(pkl_file[model_date][0].values)))  # fitting model
            # put the labels in a tensor format
            Ytr = torch.Tensor(pkl_file[model_date][1].values)
            print('first bit done')
            # force garbage collect
            nalsvm.gc.collect()
            # kernels
            KLrbf = generators.RBF_generator(Xtr, gamma=[.001, .01, .1])
            # dont need the next bit
            print('done with kernel')
            print(forward_dates)
            # base learner- use c =1 or 10
            # the c and lambda values need to be picked up by the cross-val results !
            base_learner = SVC(C=1)

            clf = EasyMKL(lam=0.2, multiclass_strategy='ova', learner=base_learner).fit(KLrbf, Ytr)
            # try ovo as
            # well
            mkl_avg = AverageMKL().fit(KLrbf, Ytr)
            print('done')
            print('the combination weights are:')
            # this bit may be redundant here and we can put it somewhere else
            for sol in clf.solution:
                print('(%d vs all): ' % sol, clf.solution[sol].weights) #dont need this loop- can make it redundant in another file
                print(len(forward_dates))

                for date in forward_dates:
                    print(date)
                    start = time.time()
                    nalsvm.logmemoryusage("Before garbage collect")
                    Xte = normalization(rescale_01(torch.Tensor(pkl_file[date][0].values)))
                    Yte = torch.Tensor(pkl_file[date][1].values)
                    try:
                        KLte = generators.RBF_generator(Xte, gamma=[.001, .01, .1])
                        print('sorted out test dates bit done')
                        nalsvm.gc.collect()
                        y_pred = clf.predict(KLte)  # predictions
                        y_score = clf.decision_function(KLte)  # rank
                        # oos_svc_predictions = defaultdict(dict)
                        accuracy = accuracy_score(Yte, y_pred)
                        accuracy_file_name = "_".join((symbol, model_date, date,alternate_label,'OOSResult.pkl'))
                        print('Accuracy score: %.3f' % accuracy)
                        mkl_dict[str(date)] = evaluate_predictions(Yte, y_pred)

#                         # average kernel as a base line

                        y_preds_average = mkl_avg.predict(KLte)  # predict the output class
                        y_scores_average = mkl_avg.decision_function(KLte)  # returns the projection on the distance vector
                        average_accuracy = accuracy_score(Yte, y_preds_average)
                        print ('Accuracy score: %.3f' % average_accuracy)
                        average_dict[str(date)] = evaluate_predictions(Yte, y_preds_average)
                    except (ValueError, TypeError, EOFError, IndexError):
                        continue

        except (ValueError, TypeError, EOFError, IndexError):
            print('problem in :', )
            # at some point for clarity we need to clean these error up.
            continue
            nalsvm.gc.collect()
            print('done too')
        results_dict['MKL_Results'][str(model_date)] = mkl_dict
        results_dict['AvgKL_Results'][str(model_date)] = average_dict
        end = time.time()
        print(f'it took {end - start} seconds!')


RSA.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/RSA.L
0
---------------> Doing Model Date: 20170706
first bit done
done with kernel
['20170707', '20170712', '20170717', '20170718', '20170719', '20170720', '20170721', '20170724', '20170727', '20170801', '20170802', '20170803', '20170804', '20170808', '20170809', '20170814', '20170816', '20170817', '20170818', '20170822', '20170823', '20170824', '20170830', '20170901', '20170904', '20170905', '20170908', '20170912', '20170913', '20170914', '20170915', '20170918', '20170919', '20170920', '20170921', '20170922', '20170925', '20170929', '20180301', '20180302', '20180404', '20180412', '20180420']
done
the combination weights are:
(-1 vs all):  tensor([0.0072, 0.0739, 0.9189])
43
20170707
Memory usage at Before garbage collect is 1101.541376mb.
sorted out test dates bit done
Accuracy score: 0.865
Acc: 86.54%
Precision: 0.75
Recall: 0.87
F1 score weighted: 0.80
F1 score macro: 0.31
F1 score micro: 0.87


/home/ak/Envs/mklpyresearch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score: 0.809
Acc: 80.93%
Precision: 0.65
Recall: 0.81
F1 score weighted: 0.72
F1 score macro: 0.30
F1 score micro: 0.81
Hamming Loss Value: 0.19
Accuracy score: 0.435
Acc: 43.48%
Precision: 0.67
Recall: 0.43
F1 score weighted: 0.51
F1 score macro: 0.28
F1 score micro: 0.43
Hamming Loss Value: 0.57
20170717
Memory usage at Before garbage collect is 1101.541376mb.
sorted out test dates bit done
20170718
Memory usage at Before garbage collect is 1101.541376mb.
sorted out test dates bit done


/home/ak/Envs/mklpyresearch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score: 0.852
Acc: 85.15%
Precision: 0.73
Recall: 0.85
F1 score weighted: 0.78
F1 score macro: 0.31
F1 score micro: 0.85
Hamming Loss Value: 0.15
Accuracy score: 0.292
Acc: 29.18%
Precision: 0.72
Recall: 0.29
F1 score weighted: 0.37
F1 score macro: 0.21
F1 score micro: 0.29
Hamming Loss Value: 0.71
20170719
Memory usage at Before garbage collect is 1101.541376mb.
sorted out test dates bit done


...

Accuracy score: 0.248
Acc: 24.81%
Precision: 0.51
Recall: 0.25
F1 score weighted: 0.20
F1 score macro: 0.24
F1 score micro: 0.25
Hamming Loss Value: 0.75
20170929
Memory usage at Before garbage collect is 1101.0662399999999mb.
sorted out test dates bit done
20180301
Memory usage at Before garbage collect is 1101.0662399999999mb.
sorted out test dates bit done
Accuracy score: 0.870
Acc: 86.96%
Precision: 0.76
Recall: 0.87
F1 score weighted: 0.81
F1 score macro: 0.31
F1 score micro: 0.87
Hamming Loss Value: 0.13


/home/ak/Envs/mklpyresearch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score: 0.665
Acc: 66.49%
Precision: 0.78
Recall: 0.66
F1 score weighted: 0.71
F1 score macro: 0.35
F1 score micro: 0.66
Hamming Loss Value: 0.34
20180302
Memory usage at Before garbage collect is 1129.71776mb.
sorted out test dates bit done
Accuracy score: 0.756
Acc: 75.63%
Precision: 0.57
Recall: 0.76
F1 score weighted: 0.65
F1 score macro: 0.29
F1 score micro: 0.76
Hamming Loss Value: 0.24


/home/ak/Envs/mklpyresearch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score: 0.520
Acc: 52.01%
Precision: 0.63
Recall: 0.52
F1 score weighted: 0.56
F1 score macro: 0.34
F1 score micro: 0.52
Hamming Loss Value: 0.48
20180404
Memory usage at Before garbage collect is 1129.71776mb.
sorted out test dates bit done
20180412
Memory usage at Before garbage collect is 1129.71776mb.
sorted out test dates bit done
20180420
Memory usage at Before garbage collect is 1129.71776mb.
sorted out test dates bit done
it took 0.05754351615905762 seconds!


In [17]:
keys =list(results_dict)
results_dict

defaultdict(dict,
            {'MKL_Results': {'20170706': defaultdict(dict,
                          {'20170707': {'accuracy': 0.87,
                            'precision': 0.75,
                            'recall': 0.87,
                            'f1- weighted': 0.8,
                            'f1- micro': 0.87,
                            'f1- macro': 0.31,
                            'Hamming Loss': 0.13},
                           '20170712': {'accuracy': 0.81,
                            'precision': 0.65,
                            'recall': 0.81,
                            'f1- weighted': 0.72,
                            'f1- micro': 0.81,
                            'f1- macro': 0.3,
                            'Hamming Loss': 0.19},
                           '20170718': {'accuracy': 0.85,
                            'precision': 0.73,
                            'recall': 0.85,
                            'f1- weighted': 0.78,
                            'f1- micro

In [18]:
list_of_keys = list(results_dict['20170706'].keys())

In [19]:
results_keys =list(results_dict.keys())

In [20]:
os.listdir('/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/')

['AAL.L', 'ECM.L', 'RDSa.L', 'RSA.L', 'SHP.L', 'STAN.L']

In [21]:
pd.DataFrame.from_dict(results_dict[results_keys[3]])

IndexError: list index out of range

In [22]:
df = pd.concat([pd.DataFrame.from_dict(value) for key, value in results_dict.items()], axis=0, keys=results_dict.keys()) 

In [26]:
df.index.values

array([('MKL_Results', '20170707'), ('MKL_Results', '20170712'),
       ('MKL_Results', '20170718'), ('MKL_Results', '20170720'),
       ('MKL_Results', '20170721'), ('MKL_Results', '20170724'),
       ('MKL_Results', '20170801'), ('MKL_Results', '20170802'),
       ('MKL_Results', '20170803'), ('MKL_Results', '20170804'),
       ('MKL_Results', '20170809'), ('MKL_Results', '20170817'),
       ('MKL_Results', '20170818'), ('MKL_Results', '20170822'),
       ('MKL_Results', '20170823'), ('MKL_Results', '20170830'),
       ('MKL_Results', '20170905'), ('MKL_Results', '20170908'),
       ('MKL_Results', '20170913'), ('MKL_Results', '20170914'),
       ('MKL_Results', '20170915'), ('MKL_Results', '20170918'),
       ('MKL_Results', '20170919'), ('MKL_Results', '20170921'),
       ('MKL_Results', '20170922'), ('MKL_Results', '20170925'),
       ('MKL_Results', '20170929'), ('MKL_Results', '20180301'),
       ('MKL_Results', '20180302'), ('MKL_Results', '20180404'),
       ('MKL_Results', '2

In [24]:
pd.DataFrame(symbol_results)

NameError: name 'symbol_results' is not defined